## Loading and rendering a URDF

To run this notebook, first install trimesh, urdfpy, and pythreejs with
```
pip install trimesh urdfpy
conda install -c conda-forge pythreejs 
```

See [urdfpy documentation](https://urdfpy.readthedocs.io/en/latest/examples/index.html).

In [ ]:
import math
import numpy as np

import urdfpy
import pythreejs
import gtsam
from IPython.display import display

In [ ]:
robot = urdfpy.URDF.load("../../models/urdfs/ur5/ur5.urdf")

In [ ]:
print([link.name for link in robot.links])

In [ ]:
print(sorted({j.name:(j.parent, j.child) for j in robot.joints}.items()))

In [ ]:
fk = robot.link_fk()
pose = fk[robot.links[4]]
print(pose)

In [ ]:
# We use collision geometry which is much simpler than visuals!
q = {'shoulder_lift_joint' : math.radians(-30), 'elbow_joint' : math.radians(60)}
visuals = robot.collision_trimesh_fk(cfg=q)

In [ ]:
def buffer_geometry_from_trimesh(mesh):
  """Converts trimesh object to pythreejs.BufferGeometry."""
  # Based on tensorflow_graphics/notebooks/threejs_visualization.py
  # However, does not work!
  # This is annoyingly complex see https://threejsfundamentals.org/threejs/lessons/threejs-custom-buffergeometry.html

  geometry = pythreejs.BufferGeometry()

  #  geometry.addAttribute('position', context.THREE.BufferAttribute.new_object(vertices, 3))
  vertices = mesh.vertices.astype(np.float32)
  position = pythreejs.BufferAttribute(vertices, normalized=False)
  geometry.attributes['position'] = position

  # geometry.setIndex(context.THREE.BufferAttribute.new_object(faces, 1))
  faces = mesh.faces.astype(np.uint32).ravel()
  index = pythreejs.BufferAttribute(faces, normalized=False)
  geometry.index = index
  
  # geometry.computeVertexNormals()
  normals = mesh.vertex_normals.astype(np.float32)
  normal = pythreejs.BufferAttribute(normals, normalized=False)
  geometry.attributes['normal'] = normal

  return geometry

In [ ]:
def geometry_from_trimesh(mesh):
  """Converts trimesh object to pythreejs.BufferGeometry."""

  geometry = pythreejs.Geometry()

  geometry.vertices = mesh.vertices.tolist()
  geometry.faces = [pythreejs.Face3.klass(list(f) + [None]*3) for f in mesh.faces.tolist()]
  geometry.normals = mesh.vertex_normals.tolist() # no effect?

  return geometry

In [ ]:
# render last occlusion geometry for debugging
geometry = geometry_from_trimesh(list(visuals)[0])
material = pythreejs.MeshBasicMaterial(color="red")
mesh = pythreejs.Mesh( geometry, material )
mesh

In [ ]:
view_width = 600
view_height = 400
camera = pythreejs.CombinedCamera(position=[-1, 0, 0], width=view_width, height=view_height)

In [ ]:
# Create scene
# gray = pythreejs.MeshStandardMaterial(color="gray", metalNess=0.2, roughness=0.8)
# blue = pythreejs.MeshStandardMaterial(color="blue", metalNess=0.2, roughness=0.8)
# gray = pythreejs.MeshPhysicalMaterial(color="red")
# blue = pythreejs.MeshPhysicalMaterial(color="green")
gray = pythreejs.MeshBasicMaterial(color="gray")
blue = pythreejs.MeshBasicMaterial(color="blue")
scene = pythreejs.Scene()
def xyzw(pose):
    q = pose.rotation().quaternion() # wxyz
    return q[1],q[2],q[3],q[0]
material = blue
for tm in visuals:
    matrix = visuals[tm] # this is the 4*4 transform, go figure
    pose = gtsam.Pose3(matrix) # convert to GTSAM so we can get quaternion
    geometry = geometry_from_trimesh(tm) # what we really want!
    mesh = pythreejs.Mesh(geometry, material,
                          position=pose.translation().tolist(), 
                          quaternion=xyzw(pose))
    scene.add(mesh)
    material = blue if material == gray else gray

In [ ]:
key_light = pythreejs.PointLight(position=[1, 1, 1], intensity=1.5)
ambient_light = pythreejs.AmbientLight(intensity=0.4)
scene.add(key_light)
scene.add(ambient_light)
scene.add(camera)
for axis, color in [([1, 0, 0],"red"),([0, 1, 0],"green"),([0, 0, 1],"blue")]:
    gz = pythreejs.LineSegmentsGeometry(positions=[[[0, 0, 0], axis]])
    mz = pythreejs.LineMaterial(linewidth=10, color=color)
    scene.add(pythreejs.LineSegments2(gz, mz))
renderer = pythreejs.Renderer(scene=scene, camera=camera, controls=[pythreejs.OrbitControls(controlling=camera)],
                    width=view_width, height=view_height)
display(renderer)

In [ ]:
from ipywidgets import interact
@interact(ortho=True)
def setOrthographic(ortho:bool):
    camera.mode = 'orthographic' if ortho else 'perspective'